```
seq 10000 300000 | xargs -P 32 -I game_id wget https://rbc.jhuapl.edu/api/games/game_id/game_history --user SomeRegret --password $RC_PASSWORD_SR -nc -O game_id.json -q

find . -empty -type f -delete

ls . | wc -l
```

In [21]:
from glob import glob
import json
from pathlib import Path

import chess
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
files = list(glob("../wgetGames/*.json"))
files[0], len(files)

('../wgetGames/228132.json', 236962)

In [55]:
with open(files[0], "rb") as f:
    data = json.load(f)
data

{'game_history': {'black_name': 'StrangeFish2',
  'capture_squares': {'false': [None, None, 45, 11, 4],
   'true': [None, None, None, None, None]},
  'fens_after_move': {'false': ['rnbqkbnr/pppp1ppp/8/4p3/8/2N5/PPPPPPPP/R1BQKBNR w KQkq e6 0 2',
    'rnbqk1nr/pppp1ppp/8/4p3/1b2N3/8/PPPPPPPP/R1BQKBNR w KQkq - 2 3',
    'rnb1k1nr/pppp1ppp/5q2/4p3/1b6/8/PPPPPPPP/R1BQKBNR w KQkq - 0 4',
    'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPPbPPPP/R1BQKBNR w KQkq - 0 5',
    'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPP1PPPP/R1BQbBNR w kq - 0 6'],
   'true': ['rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq - 1 1',
    'rnbqkbnr/pppp1ppp/8/4p3/4N3/8/PPPPPPPP/R1BQKBNR b KQkq - 1 2',
    'rnbqk1nr/pppp1ppp/5N2/4p3/1b6/8/PPPPPPPP/R1BQKBNR b KQkq - 3 3',
    'rnb1k1nr/pppp1ppp/5q2/4p3/1b6/8/PPPPPPPP/R1BQKBNR b KQkq - 1 4',
    'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPPbPPPP/R1BQKBNR b KQkq - 1 5']},
  'fens_before_move': {'false': ['rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq - 1 1',
    'rnbqkbnr/pppp1ppp/8/4p3/4N3/

In [56]:
def get_actions(history):
    actions = []
    white_senses = iter(history["senses"]["true"])
    black_senses = iter(history["senses"]["false"])
    white_moves = iter(history["requested_moves"]["true"])
    black_moves = iter(history["requested_moves"]["false"])
    try:
        while True:
            square = next(white_senses)
            square = "00" if square is None else chess.SQUARE_NAMES[square]
            actions.append(square)
            move = next(white_moves)
            move = "0000" if move is None else move["value"]
            actions.append(move)
            square = next(black_senses)
            square = "00" if square is None else chess.SQUARE_NAMES[square]
            actions.append(square)
            move = next(black_moves)
            move = "0000" if move is None else move["value"]
            actions.append(move)
    except StopIteration:
        pass
    return " ".join(actions)

get_actions(data["game_history"])

'a8 b1c3 f2 e7e5 b1 c3e4 e3 f8b4 a4 e4f6 d5 d8f6 b5 0000 c4 b4d2 g3 0000 b2 d2e1'

In [57]:
data["game_history"]

{'black_name': 'StrangeFish2',
 'capture_squares': {'false': [None, None, 45, 11, 4],
  'true': [None, None, None, None, None]},
 'fens_after_move': {'false': ['rnbqkbnr/pppp1ppp/8/4p3/8/2N5/PPPPPPPP/R1BQKBNR w KQkq e6 0 2',
   'rnbqk1nr/pppp1ppp/8/4p3/1b2N3/8/PPPPPPPP/R1BQKBNR w KQkq - 2 3',
   'rnb1k1nr/pppp1ppp/5q2/4p3/1b6/8/PPPPPPPP/R1BQKBNR w KQkq - 0 4',
   'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPPbPPPP/R1BQKBNR w KQkq - 0 5',
   'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPP1PPPP/R1BQbBNR w kq - 0 6'],
  'true': ['rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq - 1 1',
   'rnbqkbnr/pppp1ppp/8/4p3/4N3/8/PPPPPPPP/R1BQKBNR b KQkq - 1 2',
   'rnbqk1nr/pppp1ppp/5N2/4p3/1b6/8/PPPPPPPP/R1BQKBNR b KQkq - 3 3',
   'rnb1k1nr/pppp1ppp/5q2/4p3/1b6/8/PPPPPPPP/R1BQKBNR b KQkq - 1 4',
   'rnb1k1nr/pppp1ppp/5q2/4p3/8/8/PPPbPPPP/R1BQKBNR b KQkq - 1 5']},
 'fens_before_move': {'false': ['rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq - 1 1',
   'rnbqkbnr/pppp1ppp/8/4p3/4N3/8/PPPPPPPP/R1BQKBNR b KQkq - 1 

In [28]:
data = []
for filename in tqdm(list(glob("../wgetGames/*.json"))):
    with open(filename, "rb") as f:
        game = json.load(f)["game_history"]
    data.append((
        int(filename.split("/")[-1][:-5]),
        game["white_name"],
        game["black_name"],
        game["winner_color"],
        game["win_reason"]["value"] == "TIMEOUT",
        get_actions(game),
    ))
df = pd.DataFrame(data, columns=["id", "white", "black", "winner", "timeout", "actions"]).set_index("id")

  0%|          | 0/236962 [00:00<?, ?it/s]

In [29]:
df

,white,black,winner,timeout,actions
id,,,,,
228132,attacker,StrangeFish2,False,False,a8 b1c3 f2 e7e5 b1 c3e4 e3 f8b4 a4 e4f6 d5 d8f...
86166,StrangeFish,DynamicEntropy,False,False,00 e2e4 g2 e7e5 g6 g1f3 c2 b8c6 e7 f3e5 d4 c6e...
116936,Zugzwang,StrangeFish2,True,False,e4 g2g3 d3 c7c5 d4 h2h3 e2 0000 e4 g1f3 d5 b8a...
57896,guest-22108,random,False,True,a1 h2h3 f8 e7e5 g5 h1h2 f1 f8e7 c7 h2h1 g4 a7a...
203262,Oracle,Autobot,True,False,e2 e2e4 b3 a7a6 c7 g1f3 f2 e7e6 g7 d2d4 f2 f8b...
...,...,...,...,...,...
112777,StrangeFish,Zugzwang,False,False,00 e2e4 e3 g7g6 g5 c2c3 f6 f8g7 f7 d2d4 e6 d7d...
54112,TOEFL,random,False,False,h5 g2g4 d2 a7a6 f3 d2d3 e4 h7h6 b1 g4f5 f5 d7e...
231042,StrangeFish,Oracle,True,True,00 c2c4 b3 e7e5 c7 d2d4 b3 b8c6 e7 d4e5 g2 c6e...


In [30]:
df.to_pickle("histories.pkl")

In [31]:
df.to_csv("histories.csv")

In [32]:
from dask.distributed import Client, progress

In [33]:
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:53755 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [34]:
import dask
import dask.bag as db

In [45]:
def parse(path_and_text):
    text, path = path_and_text
    game = json.loads(text)["game_history"]
    game["id"] = int(path.split("/")[-1][:-5])
    return game

b = db.read_text('../wgetGames/*.json', include_path=True).map(parse)
b

dask.bag<parse, npartitions=236962>

In [46]:
b.take(2)

({'black_name': 'apltest54',
  'capture_squares': {'false': [], 'true': []},
  'fens_after_move': {'false': [], 'true': []},
  'fens_before_move': {'false': [], 'true': []},
  'requested_moves': {'false': [], 'true': []},
  'sense_results': {'false': [], 'true': []},
  'senses': {'false': [], 'true': []},
  'taken_moves': {'false': [], 'true': []},
  'type': 'GameHistory',
  'white_name': 'apltest39',
  'win_reason': {'type': 'WinReason', 'value': 'TIMEOUT'},
  'winner_color': False,
  'id': 10000},)

In [47]:
def flatten(game):
    return {
        "id": game["id"],
        "white": game["white_name"],
        "black": game["black_name"],
        "winner": game["winner_color"],
        "timeout": game["win_reason"]["value"] == "TIMEOUT",
        "actions": get_actions(game),
    }

In [48]:
df = b.map(flatten).to_dataframe()

In [49]:
df.head()

,id,white,black,winner,timeout,actions
0,10000,apltest39,apltest54,False,True,


In [51]:
df.compute()

,id,white,black,winner,timeout,actions
0,10000,apltest39,apltest54,False,True,
0,100000,StrangeFish,random,True,False,00 e2e4 b5 c7c5 g7 g1f3 g4 c5c4 e7 d2d4 c8 h7h...
0,100001,random,Oracle,False,False,d7 b2a3 c2 d7d5 a1 b2c3 b4 e7e5 a3 f2g3 f2 g8f...
0,100002,DynamicEntropy,random,True,False,00 e2e4 g3 b7a6 g6 g1f3 b3 g7h6 c7 b1c3 h4 g7f...
0,100003,Marmot,StrangeFish,False,False,g2 g1f3 e2 d7d5 d5 e2e4 f4 d5e4 c4 f1b5 c4 c7c...
...,...,...,...,...,...,...
0,99995,DynamicEntropy,StrangeFish,False,False,00 e2e4 e4 e7e5 b6 b1c3 e2 g8f6 f7 g1f3 e2 f6e...
0,99996,StrangeFish,Oracle,True,False,00 e2e4 b3 d7d5 g7 g1f3 c4 e7e5 e7 e4d5 f2 e5e...
0,99997,DynamicEntropy,Oracle,True,False,00 e2e4 c2 d7d5 g7 g1f3 b4 e7e5 c7 e4d5 e2 g8f...
0,99998,DynamicEntropy,Marmot,True,False,00 e2e4 e2 b8c6 f7 d2d4 e3 b7b5 c7 d4d5 c2 e7e...


In [53]:
df = df.set_index("id")

KeyboardInterrupt: 

In [ ]:
df.to_pickle("dask_histories.pkl")
df.to_csv("dask_histories.csv")

In [58]:
client.close()